# Agente con Middleware de PII



## Características:
- **Redacción de Email**: Elimina correos electrónicos de la entrada del usuario
- **Enmascaramiento de Tarjetas de Crédito**: Oculta números de tarjetas de crédito
- **Bloqueo de Claves API**: Bloquea solicitudes que contengan claves API

In [10]:
import os
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()


True

## Importar Librerías Requeridas

In [11]:
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import get_buffer_string

## Definir Herramientas Personalizadas


In [12]:
@tool
def customer_service_tool(query: str) -> str:
    """Maneja consultas de servicio al cliente y proporciona asistencia."""
    return f"Procesando consulta de servicio al cliente: {query}"

@tool
def email_tool(message: str) -> str:
    """Envía una notificación o mensaje por correo electrónico."""
    return f"Se enviaría un correo con el mensaje: {message}"

# Mostrar herramientas disponibles
print("Herramientas disponibles:")
print(f"1. {customer_service_tool.name}: {customer_service_tool.description}")
print(f"2. {email_tool.name}: {email_tool.description}")

Herramientas disponibles:
1. customer_service_tool: Maneja consultas de servicio al cliente y proporciona asistencia.
2. email_tool: Envía una notificación o mensaje por correo electrónico.



1. **Redactar** - Elimina correos electrónicos completamente
2. **Enmascarar** - Reemplaza números de tarjetas de crédito con asteriscos
3. **Bloquear** - Genera un error si se detectan claves API

In [13]:
agent = create_agent(
    model="gpt-4o",
    tools=[customer_service_tool, email_tool],
    middleware=[
        # Redactar emails en la entrada del usuario antes de enviar al modelo
        PIIMiddleware(
            "email",
            strategy="redact",
            apply_to_input=True,
        ),
        # Enmascarar tarjetas de crédito en la entrada del usuario
        PIIMiddleware(
            "credit_card",
            strategy="mask",
            apply_to_input=True,
        ),
        # Bloquear claves API - generar error si se detectan
        PIIMiddleware(
            "api_key",
            detector=r"sk-[a-zA-Z0-9]{32}",
            strategy="block",
            apply_to_input=True,
        ),
    ],
)


## Probar el Agente

### Ejemplo 1: Manejo de Email y Tarjeta de Crédito

In [14]:
# Prueba con email y tarjeta de crédito
result = agent.invoke({
    "messages": [
        {
            "role": "user", 
            "content": "Mi correo es juan.perez@ejemplo.com y mi tarjeta es 5105-1051-0510-5100"
        }
    ]
})

print(get_buffer_string(
    result["messages"],
    human_prefix="👤 Usuario",
    ai_prefix="🤖 Asistente"
))

👤 Usuario: Mi correo es [REDACTED_EMAIL] y mi tarjeta es ****-****-****-5100
🤖 Asistente: Lo siento, pero no puedo ayudarte con eso. Si tienes alguna pregunta o necesitas asistencia, no dudes en preguntarme.


### Ejemplo 2: Consulta de Servicio al Cliente

In [15]:
# Prueba de funcionalidad de servicio al cliente
result2 = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Necesito ayuda con mi cuenta. Contáctame en soporte@ejemplo.com"
        }
    ]
})

print("\n=== Conversación ===")
print(get_buffer_string(
    result2["messages"],
    human_prefix="👤 Usuario",
    ai_prefix="🤖 Asistente"
))


=== Conversación ===
👤 Usuario: Necesito ayuda con mi cuenta. Contáctame en [REDACTED_EMAIL]
🤖 Asistente: He eliminado la dirección de correo electrónico por razones de privacidad. Dígame, ¿en qué necesita ayuda con su cuenta? Así podré asistirle mejor o dirigir su consulta al departamento adecuado.


### Ejemplo 3: Prueba de Bloqueo de Clave API (Esto debería generar un error)

In [16]:
# Prueba de bloqueo de API keys - esto debería generar un error
try:
    result3 = agent.invoke({
        "messages": [
            {
                "role": "user",
                "content": "Aquí está mi clave de API: sk-abcdefghijklmnopqrstuvwxyz123456"
            }
        ]
    })
    print("\n=== Respuesta del Agente ===")
    print(result3)
except Exception as e:
    print("\n=== Error (Esperado) ===")
    print(f"La clave API fue bloqueada: {e}")


=== Error (Esperado) ===
La clave API fue bloqueada: Detected 1 instance(s) of api_key in text content
